In [26]:
<!DOCTYPE html>
<html>
<head>
    <title>Guess the Number Game</title>
    <link rel="stylesheet" type="text/css" href="style.css">
</head>
<body>
    <h1>Guess the Number Game</h1>
    <p>Guess a number between 1 and 100:</p>
    <input type="text" id="guessField">
    <input type="submit" value="Submit Guess" class="guessSubmit">
    <p class="message"></p>

    <script src="script.js"></script>
</body>
</html>

<IPython.core.display.Javascript object>